In [1]:
import mlflow
from model.utils.data_preprocess import preprocess_data, split_data
from model.models.models import XGBModel
import pandas as pd 
import json

2024-10-06 14:02:55.753625: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-06 14:02:55.761837: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-06 14:02:55.838759: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-06 14:02:55.932623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 14:02:56.006861: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
mlflow.login()


2024/10/06 14:03:11 INFO mlflow.utils.credentials: No valid Databricks credentials found, please enter your credentials...
2024/10/06 14:03:30 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://community.cloud.databricks.com.


In [3]:
mlflow.set_tracking_uri("databricks")


In [4]:
df = preprocess_data(pd.read_csv('/home/miza/housepricesml/model/data/train.csv'))
X_train, X_test, y_train, y_test = split_data(df)

In [5]:
with open('/home/miza/housepricesml/best_params.json', "r") as f:
    params = json.load(f)
model = XGBModel(**params)
model.fit(X_train, y_train)
r2 = model.evaluate(X_test, y_test)


In [13]:
mlflow.set_experiment("/Users/michal.zareba@softwaremill.pl/houseprices")
with mlflow.start_run():
    for key,value in params.items():
        mlflow.log_param(key, value)
    mlflow.log_metric("r2", r2)
    mlflow.set_tag("Training info", "XGB regressor for house prices")
    mlflow.log_artifact("model/data/train.csv")
    # dissabled due to Databricks Community Edition limitations
    # signature = infer_signature(X_train, model.predict(X_train))
    # model_info = mlflow.sklearn.log_model(
    #     sk_model= model,
    #     artifact_path="houseprices_model",
    #     signature=signature,
    #     input_example=X_train,
    #     registered_model_name="houseprices_model"
    #     )


2024/10/06 14:10:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run thoughtful-crow-86 at: https://community.cloud.databricks.com/ml/experiments/3736160771961302/runs/4ccc673c5b224f6a9e442873d0fd2eef.
2024/10/06 14:10:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/3736160771961302.
